In [22]:
import requests
import json
from typing import List, Dict, Any, Optional

In [3]:
# AIRBYTE_BASE_URL = "http://ec2-54-78-209-246.eu-west-1.compute.amazonaws.com/api" # DEV
AIRBYTE_BASE_URL = "http://ec2-52-16-71-181.eu-west-1.compute.amazonaws.com/api" # PROD


In [25]:
ENV = "prod"

In [4]:
def _get_workspaces() -> dict:
    url = f"{AIRBYTE_BASE_URL}/v1/workspaces/list"
    response = requests.post(url)

    if response.status_code == 200:
        return response.json()
    raise ValueError(response.json()["message"])


def _get_workspace_id() -> str:
    workspaces = _get_workspaces().get("workspaces", [])
    if len(workspaces) > 0:
        return workspaces[0].get("workspaceId", None)
    return None


In [5]:
def find_all_connections() -> list:
    payload = {"workspaceId": _get_workspace_id()}
    url = f"{AIRBYTE_BASE_URL}/v1/connections/list"
    response = requests.post(url, json=payload)

    if response.status_code != 200:
        raise ValueError(response.json()["message"])

    connections = response.json().get("connections", [])
    return [connection for connection in connections]

In [6]:
all_connections = find_all_connections()

In [7]:
print(type(all_connections))
print(len(all_connections))

<class 'list'>
733


In [ ]:
all_connections[:3]

In [24]:
conn_event = "ListFinancialEvents"

# Initialize an empty list to store the filtered results
data = []

# Iterate through each element in the list
for element in all_connections:
    if conn_event in element['name']:
        data.append({
            'name': element['name'],
            'sourceId': element['sourceId'],
            'connectionId': element['connectionId'],
            'destinationId': element['destinationId'],
        })

# Convert the list of dictionaries to JSON format
data_json = json.dumps(data, indent=2)

print(data_json)

[
  {
    "name": "Custom Amazon Seller Partner - oliver_james (EU - UK): ListFinancialEvents >> S3 heroes-dl-raw-prod Bucket",
    "sourceId": "8d4e35ec-3e66-4c61-85d2-384d4a69a41a",
    "connectionId": "09fb98ec-8944-4818-a223-ad3e2cf4bb9d",
    "destinationId": "0a044861-e642-4460-843c-44f4d20525c7"
  },
  {
    "name": "Custom Amazon Seller Partner - baby_uma (AU - AU): ListFinancialEvents >> S3 heroes-dl-raw-prod Bucket",
    "sourceId": "4be2c06e-26bc-4909-9d6e-1d097964addc",
    "connectionId": "1158919d-86d4-4850-884c-f741e0f49a75",
    "destinationId": "0a044861-e642-4460-843c-44f4d20525c7"
  },
  {
    "name": "Custom Amazon Seller Partner - good_natured_sun_organics (EU - UK): ListFinancialEvents >> S3 heroes-dl-raw-prod Bucket",
    "sourceId": "13fab76e-5e92-4eb4-af55-ff9c0ea9ca1a",
    "connectionId": "1fc0d87f-bcf1-421b-b96e-643e50563201",
    "destinationId": "0a044861-e642-4460-843c-44f4d20525c7"
  },
  {
    "name": "Custom Amazon Seller Partner - trunki_us (NA - US):

In [21]:
# Write the JSON data to a file
with open(f"airbyte_financial_events_connections_{ENV}.json", "w") as json_file:
    json_file.write(data_json)

print('JSON data has been written to filtered_data.json')

JSON data has been written to filtered_data.json
